In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from Vis import ClassificationVis, LabelVis
from Evals import *
from Data_Provider import *
from Util import ClassificationUtils as util
from Util import DataUtils as dutil

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier


In [3]:
# #important configs
# configs = dutil.load_configs()
# print(configs) #is this what we want?

In [4]:
# try:
#     x,y,x_ev,y_ev = dutil.get_data_from_file(configs)
# except:
#     raise NameError('File does not exist')


In [5]:
# print(x.shape)
# print(y.shape)
# print(x_ev.shape)
# print(y_ev.shape)

# LabelVis.plot_happy_bars(y,y_ev)


In [15]:
#Run classifier with cross-validation and plot ROC curves
def calc_results_and_save(x,y,x_ev,y_ev, configs):
    cv = StratifiedKFold(n_splits=10,shuffle=True,random_state = 1)
    results = pd.DataFrame(columns=('C','Gamma','AVG PR'))#,'AUC ROC'))
    cs = np.random.uniform(0,5,50)
    gammas = 2**(np.random.choice(18,50)-15.)

    for idx,(g,c) in enumerate(zip(gammas,cs)):
        classifier = svm.SVC(cache_size=16000,C=c,kernel='rbf', probability = True,gamma = g, random_state=5)
        auc_pr = util.get_auc_score(classifier, cv, x, y, go_after_pr=True)
        #auc_roc =util.get_auc_score(classifier, cv, x, y, go_after_pr=False)
        results.loc[idx] = [c,g,auc_pr]
        print('Gamma = %.6f, C= %2.2f, Avg PR: %.4f' % (g,c,auc_pr))
    dutil.save_results(results,configs, '_rbf')
    return results

def get_best_hyperparas(results):
    best_choice = util.get_best_hyperparas_results(results, col='Avg PR') #what were the best hyperparas?
    print(best_choice)
    best_c,best_g =best_choice[:2]
    return best_c, best_g
    
def vis_results(x,y, x_ev, y_ev, best_c, best_g):
    ###using hyperpara found, evaluate and get pretty plots
    classifier = svm.SVC(C=best_c, kernel='rbf', probability=True,gamma = best_g,random_state=5)
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    best_thr = util.get_optimal_threshold(classifier, cv, x, y, go_after_pr=True) # get threshold using cv
    #for thr in np.linspace(0,1,10):
    print(best_thr)
    y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)

    print(prec_tr, recall_tr,f1_tr)
    print(prec_ev, recall_ev,f1_ev)

    


    ClassificationVis.conf_mat(y_pred,y)
    ClassificationVis.conf_mat(y_pred_ev,y_ev)
    plt.show()
    classifier.fit(x,y)
    ClassificationVis.plot_roc(x,y,classifier, 'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
    ClassificationVis.plot_roc(x_ev,y_ev,classifier, 'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
    ClassificationVis.plot_pr_curve(x,y,classifier,'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
    ClassificationVis.plot_pr_curve(x_ev,y_ev,classifier,'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))



In [16]:
%matplotlib inline

In [17]:
files = [f for f in os.listdir('/home/emil/OpenMindv2/data') if f.endswith('hdf')]
provider = DataProvider()
for file in files:
    print('File',file)
    for cut in [.1,.2,.4]:
        print('Cut=',cut)
        configs = dutil.generate_configs_from_file(file, cut)
        x,y,x_ev,y_ev = provider.get_data(configs)
        print(x.shape)
        res = calc_results_and_save(x,y,x_ev,y_ev, configs)
#         a,b = get_best_hyperparas(res)
#         vis_results(x,y, x_ev, y_ev,a,b)


File ws_30_str_5_tr_s_0_e_30000_ev_s_30000_e_35000_expvar_90.hdf
Cut= 0.1
Loading Data from File..done
Doing cutoff
(5182, 99)


KeyboardInterrupt: 